#SoMe Topic Modeling Notebook | Release canvas 1 📖

In [10]:
# Installations
import sys
if 'google.colab' in sys.modules:
    !pip install emoji --upgrade
    !pip install pandas-profiling==2.*
    !pip install plotly==4.*
    !python -m spacy download en_core_web_lg
    !pip install pyldavis
    !pip install gensim
    !pip install squarify==0.4.3

Requirement already up-to-date: emoji in /usr/local/lib/python3.6/dist-packages (0.5.4)
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
# Required Libraries

#Base and Cleaning 
import json
import requests
import pandas as pd
import numpy as np
import emoji
import regex
import re
import string
import squarify
from collections import Counter

#Visualizations
import plotly.express as px
import seaborn as sns
import matplotlib as plt 
import pyLDAvis.gensim

#Natural Language Processing (NLP)
import spacy
from spacy.tokenizer import Tokenizer
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS as SW
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)

##Data Cleaning 🧹

In [12]:
# Loading the JSON file 
url_elon = 'https://raw.githubusercontent.com/Lambda-School-Labs/social-media-strategy-ds/feature/topic/python_notebooks/elonmusk_followers_english.json'
url_dutchbros = 'https://raw.githubusercontent.com/Lambda-School-Labs/social-media-strategy-ds/feature/topic/python_notebooks/dutchbros_followers.json'
df = requests.get(url_dutchbros).json()

# Converting the dataset to pandas DataFrame and renaming the columns 
df = pd.DataFrame(df.values())
df = df.rename(columns={0:'original_tweets'})

#Removing emojies from text and and puttin them in a new column
#Refrence 1 : https://stackoverflow.com/questions/43146528/how-to-extract-all-the-emojis-from-text
#Refrence 2 : https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
#Refrence 3 : https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python

### Question: How to combine all functions into one 
def extract_emoji(text):
    '''
    Extracts emojies from text
    '''
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)
    return emoji_list

def emoji_free_text(text):
    '''
    Cleans text from emojies
    '''
    emoji_list_1 = [c for c in text if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list_1)])
    return clean_text

def url_free_text(text):
    '''
    Cleans text from urls
    '''
    text = re.sub(r'http\S+', '', text)
    return text

#Create a new column with emojies from each tweet
df['tweet_emojies'] = df['original_tweets'].apply(lambda x : extract_emoji(x))

#Create a new column with cleaned tweets from emojies
df['emoji_free_tweets'] = df['original_tweets'].apply(lambda x : emoji_free_text(x))

#Create a new column with url free tweets
df['url_free_tweets'] = df['emoji_free_tweets'].apply(lambda x : url_free_text(x))

df.head()

,original_tweets,tweet_emojies,emoji_free_tweets,url_free_tweets
0,Flynn was fucking railroaded. https://t.co/4aZ...,[],Flynn was fucking railroaded. https://t.co/4aZ...,Flynn was fucking railroaded.
1,President @realDonaldTrump talks about Opening...,[],President @realDonaldTrump talks about Opening...,President @realDonaldTrump talks about Opening...
2,"#Obamagate Comey magic: Flynn calls ""turned up...",[],"#Obamagate Comey magic: Flynn calls ""turned up...","#Obamagate Comey magic: Flynn calls ""turned up..."
3,Matt Gaetz predicts President Trump will pardo...,[],Matt Gaetz predicts President Trump will pardo...,Matt Gaetz predicts President Trump will pardo...
4,We're in the 21st century Joe... https://t.co/...,[],We're in the 21st century Joe... https://t.co/...,We're in the 21st century Joe...


##Topic Modeling ㊙️

###Tokenizing 🕵🏻‍♂

In [0]:
# Load spacy
# Make sure to restart the runtime after running installations and libraries tab
nlp = spacy.load('en_core_web_lg')

In [14]:
"""
Import Gensim and Wordcloud to use their stopwords as well and use the combined stopwords of ALL as the variable:
ALL_STOP_WORDS
"""

# Tokenizer
tokenizer = Tokenizer(nlp.vocab)


# Custom stopwords
custom_stopwords = ['hi','\n','\n\n', '&amp;', ' ', '.', '-', 'got', "it's", 'it’s', "i'm", 'i’m', 'im', 'want', 'like', '$', '@']

# Customize stop words by adding to the default list
STOP_WORDS = nlp.Defaults.stop_words.union(custom_stopwords)

# ALL_STOP_WORDS = spacy + gensim + wordcloud
ALL_STOP_WORDS = STOP_WORDS.union(SW).union(stopwords)


tokens = []

for doc in tokenizer.pipe(df['url_free_tweets'], batch_size=500):
    doc_tokens = []    
    for token in doc: 
        if token.text.lower() not in STOP_WORDS:
            doc_tokens.append(token.text.lower())   
    tokens.append(doc_tokens)

# Makes tokens column
df['tokens'] = tokens

# View df
df

,original_tweets,tweet_emojies,emoji_free_tweets,url_free_tweets,tokens
0,Flynn was fucking railroaded. https://t.co/4aZ...,[],Flynn was fucking railroaded. https://t.co/4aZ...,Flynn was fucking railroaded.,"[flynn, fucking, railroaded.]"
1,President @realDonaldTrump talks about Opening...,[],President @realDonaldTrump talks about Opening...,President @realDonaldTrump talks about Opening...,"[president, @realdonaldtrump, talks, opening, ..."
2,"#Obamagate Comey magic: Flynn calls ""turned up...",[],"#Obamagate Comey magic: Flynn calls ""turned up...","#Obamagate Comey magic: Flynn calls ""turned up...","[#obamagate, comey, magic:, flynn, calls, ""tur..."
3,Matt Gaetz predicts President Trump will pardo...,[],Matt Gaetz predicts President Trump will pardo...,Matt Gaetz predicts President Trump will pardo...,"[matt, gaetz, predicts, president, trump, pard..."
4,We're in the 21st century Joe... https://t.co/...,[],We're in the 21st century Joe... https://t.co/...,We're in the 21st century Joe...,"[we're, 21st, century, joe...]"
...,...,...,...,...,...
18518,After a great phone call with the @ASUFootball...,"[🔥, ☀️, 🔥, ☀️, 🔥, ☀️, 🔥, ☀️, 🔥, ☀️, 🔥]",After a great phone call with the @ASUFootball...,After a great phone call with the @ASUFootball...,"[great, phone, @asufootball, coach’s, humbled,..."
18519,After a great zoom meeting with the @ASUFootba...,[],After a great zoom meeting with the @ASUFootba...,After a great zoom meeting with the @ASUFootba...,"[great, zoom, meeting, @asufootball, staff,, e..."
18520,Morning 💗 https://t.co/uu7MhenMOf,[💗],Morning https://t.co/uu7MhenMOf,Morning,[morning]
18521,Always be careful who you open up to. Only a f...,[],Always be careful who you open up to. Only a f...,Always be careful who you open up to. Only a f...,"[careful, open, to., actually, care,, curious.]"


###Lemmatization🇬🇧

In [0]:
# Make tokens a string again
# credit : https://stackoverflow.com/questions/45306988/column-of-lists-convert-list-to-string-as-a-new-column
df['tokens_back_to_text'] = [' '.join(map(str, l)) for l in df['tokens']]

def get_lemmas(text):
    '''Used to lemmatize the processed tweets'''
    lemmas = []
    
    doc = nlp(text)
    
    # Something goes here :P
    for token in doc: 
        if ((token.is_stop == False) and (token.is_punct == False)) and (token.pos_ != 'PRON'):
            lemmas.append(token.lemma_)
    
    return lemmas

df['lemmas'] = df['tokens_back_to_text'].apply(get_lemmas)

# Make lemmas a string again
df['lemmas_back_to_text'] = [' '.join(map(str, l)) for l in df['lemmas']]
# df[['original_tweet', 'lemmas_back_to_text']]

##

In [16]:
# Tokenizer
tokenizer = Tokenizer(nlp.vocab)

# Tokenizer function
def tokenize(text):
    """
    Parses a string into a list of semantic units (words)
    Args:
        text (str): The string that the function will tokenize.
    Returns:
        list: tokens parsed out
    """
    # Removing url's
    pattern = r"http\S+"
    
    tokens = re.sub(pattern, "", text) # https://www.youtube.com/watch?v=O2onA4r5UaY
    tokens = re.sub('[^a-zA-Z 0-9]', '', text)
    tokens = re.sub('[%s]' % re.escape(string.punctuation), '', text) # Remove punctuation
    tokens = re.sub('\w*\d\w*', '', text) # Remove words containing numbers
    tokens = re.sub('@', '', text) # Remove @
    tokens = tokens.strip(',') # TESTING THIS LINE
    tokens = tokens.strip('?') # TESTING THIS LINE
    tokens = tokens.strip('!') # TESTING THIS LINE
    tokens = tokens.strip("'") # TESTING THIS LINE
    tokens = tokens.strip(".") # TESTING THIS LINE
    tokens = tokens.strip("$") # TESTING THIS LINE
    tokens = tokens.strip("@") # TESTING THIS LINE

    tokens = tokens.lower().split() # Make text lowercase and split it
    
    return tokens

# Apply tokenizer
df['lemma_tokens'] = df['lemmas_back_to_text'].apply(tokenize)

# View those tokens (the 4th column)
df

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:19: DeprecationW

,original_tweets,tweet_emojies,emoji_free_tweets,url_free_tweets,tokens,tokens_back_to_text,lemmas,lemmas_back_to_text,lemma_tokens
0,Flynn was fucking railroaded. https://t.co/4aZ...,[],Flynn was fucking railroaded. https://t.co/4aZ...,Flynn was fucking railroaded.,"[flynn, fucking, railroaded.]",flynn fucking railroaded.,"[flynn, fucking, railroaded]",flynn fucking railroaded,"[flynn, fucking, railroaded]"
1,President @realDonaldTrump talks about Opening...,[],President @realDonaldTrump talks about Opening...,President @realDonaldTrump talks about Opening...,"[president, @realdonaldtrump, talks, opening, ...",president @realdonaldtrump talks opening ameri...,"[president, @realdonaldtrump, talks, opening, ...",president @realdonaldtrump talks opening america,"[president, realdonaldtrump, talks, opening, a..."
2,"#Obamagate Comey magic: Flynn calls ""turned up...",[],"#Obamagate Comey magic: Flynn calls ""turned up...","#Obamagate Comey magic: Flynn calls ""turned up...","[#obamagate, comey, magic:, flynn, calls, ""tur...","#obamagate comey magic: flynn calls ""turned up...","[obamagate, comey, magic, flynn, call, turn, f...",obamagate comey magic flynn call turn find ova...,"[obamagate, comey, magic, flynn, call, turn, f..."
3,Matt Gaetz predicts President Trump will pardo...,[],Matt Gaetz predicts President Trump will pardo...,Matt Gaetz predicts President Trump will pardo...,"[matt, gaetz, predicts, president, trump, pard...",matt gaetz predicts president trump pardon rog...,"[matt, gaetz, predict, president, trump, pardo...",matt gaetz predict president trump pardon roge...,"[matt, gaetz, predict, president, trump, pardo..."
4,We're in the 21st century Joe... https://t.co/...,[],We're in the 21st century Joe... https://t.co/...,We're in the 21st century Joe...,"[we're, 21st, century, joe...]",we're 21st century joe...,"[21st, century, joe]",21st century joe,"[21st, century, joe]"
...,...,...,...,...,...,...,...,...,...
18518,After a great phone call with the @ASUFootball...,"[🔥, ☀️, 🔥, ☀️, 🔥, ☀️, 🔥, ☀️, 🔥, ☀️, 🔥]",After a great phone call with the @ASUFootball...,After a great phone call with the @ASUFootball...,"[great, phone, @asufootball, coach’s, humbled,...",great phone @asufootball coach’s humbled bless...,"[great, phone, @asufootball, coach, humble, bl...",great phone @asufootball coach humble blessed ...,"[great, phone, asufootball, coach, humble, ble..."
18519,After a great zoom meeting with the @ASUFootba...,[],After a great zoom meeting with the @ASUFootba...,After a great zoom meeting with the @ASUFootba...,"[great, zoom, meeting, @asufootball, staff,, e...","great zoom meeting @asufootball staff, extreme...","[great, zoom, meeting, @asufootball, staff, ex...",great zoom meeting @asufootball staff extremel...,"[great, zoom, meeting, asufootball, staff, ext..."
18520,Morning 💗 https://t.co/uu7MhenMOf,[💗],Morning https://t.co/uu7MhenMOf,Morning,[morning],morning,[morning],morning,[morning]
18521,Always be careful who you open up to. Only a f...,[],Always be careful who you open up to. Only a f...,Always be careful who you open up to. Only a f...,"[careful, open, to., actually, care,, curious.]","careful open to. actually care, curious.","[careful, open, actually, care, curious]",careful open actually care curious,"[careful, open, actually, care, curious]"


###id2word 📒

In [17]:
# Create a id2word dictionary
id2word = Dictionary(df['lemma_tokens'])
print(len(id2word))

24941


In [18]:
# Filtering Extremes
id2word.filter_extremes(no_below=2, no_above=.99)
print(len(id2word))

10190


###Corpus Object & Generating Topics 📚

In [0]:
# Creating a corpus object 
corpus = [id2word.doc2bow(d) for d in df['lemma_tokens']]

In [20]:
# Instantiating a LDA model 
model = LdaMulticore(corpus=corpus, num_topics=5, id2word=id2word, workers=12, passes=5)

Streaming output truncated to the last 5000 lines.

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.6/dist-pa

In [0]:
# Filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in model.print_topics()]

In [0]:
# Create Topics
topics = [' '.join(t[0:5]) for t in words]

In [23]:
# Getting the topics
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
time love feel know think

------ Topic 1 ------
follow come retweet thank tweet

------ Topic 2 ------
go new thing feel people

------ Topic 3 ------
people good love think know

------ Topic 4 ------
fuck find time think shit



###### Topic Distance Visualization 📈

In [24]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.001998  0.018169       1        1  24.156822
1     -0.038189 -0.029744       2        1  23.428261
2     -0.047510 -0.058056       3        1  18.546490
0     -0.019335  0.089506       4        1  18.178421
4      0.107032 -0.019875       5        1  15.690003, topic_info=       Term        Freq       Total Category  logprob  loglift
177  follow  422.000000  422.000000  Default  30.0000  30.0000
97     fuck  305.000000  305.000000  Default  29.0000  29.0000
184  people  968.000000  968.000000  Default  28.0000  28.0000
89     feel  423.000000  423.000000  Default  27.0000  27.0000
257    love  777.000000  777.000000  Default  26.0000  26.0000
..      ...         ...         ...      ...      ...      ...
79   friend   71.520325  421.077942   Topic5  -5.7528   0.0793
118    need   77.114830  527.523376   Topic5  -5.6775  -0.0708
107     day   76.740471  604.668945   Topic5  -5.6823  -0.2121
276     say   62.793236  533.463684   Topic5  -5.8829  -0.2874
184  people   66.165001  968.232483   Topic5  -5.8306  -0.8312

[453 rows x 6 columns], token_table=      Topic      Freq Term
term                      
836       1  0.034033    $
836       2  0.494783    $
836       3  0.172781    $
836       4  0.282733    $
836       5  0.015707    $
...     ...       ...  ...
3652      2  0.912257    ❈
4372      1  0.887352    ➤
5802      1  0.922485    ⠀
5802      2  0.035480    ⠀
5802      3  0.035480    ⠀

[1175 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 3, 1, 5])

##Make a pipeline 

In [0]:
from sklearn.compose import make_column_transformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
